In [3]:
from clase_sms_ccd import Clase_SMS
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

excel_rutas = "./RUTAS.xlsx"
df_rutas = pd.read_excel(excel_rutas)
fecha_hoy = datetime.today()
fecha_ayer = fecha_hoy - timedelta(days=1)
fecha_ayer = fecha_ayer.strftime("%Y%m%d")
fecha_hoy = datetime.today().strftime("%Y%m%d")
fecha_hoy_txt = datetime.today().strftime("%d.%m.%Y")
ruta_zfir60 = df_rutas["RUTA"][0]
ruta_modelo = df_rutas["RUTA"][1]
ruta_dacxanalista = df_rutas["RUTA"][2]

reporte = Clase_SMS(fecha_hoy, fecha_ayer, fecha_hoy_txt, ruta_zfir60, ruta_modelo, ruta_dacxanalista)

In [ ]:
reporte.traer_archivos()
reporte.preparar_zfir60()
reporte.exportar_deudores_sap()

In [ ]:
reporte.preparar_fbl5n()
reporte.preparar_recaudacion()

In [ ]:
reporte.exportar_archivos_txt()

In [ ]:
#---------------------------------------- FIN DEL PROGRAMA ----------------------------------------#

In [ ]:
#---------------------------------------- EXPORTAR TXT ----------------------------------------#

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_dacxanalista = pd.read_excel(ruta_dacxanalista, sheet_name="Base_NUEVA")
df_dacxanalista = df_dacxanalista[["DEUDOR", "TIPO_DAC", "ESTADO"]]
estados = ["OPERATIVO CON MOVIMIENTO", "OPERATIVO SIN MOVIMIENTO"]
lista_tipo_dac_no_validos = ["PVA", "TARJETERO", "RED", "RECARGA FFVV", "PROVEEDOR", "CROSSBORDER", "CDR", "CACE", "AGENTE TELMEX", "AGENTE TELMEX  / RED", "DAC RURAL"]
df_dacxanalista = df_dacxanalista[df_dacxanalista["ESTADO"].isin(estados)]
df_dacxanalista = df_dacxanalista[~df_dacxanalista["TIPO_DAC"].isin(lista_tipo_dac_no_validos)]

# Valida que los deudores de la base se encuentren operativos en el dacxanalista
deudores_dacxanalista = df_dacxanalista["DEUDOR"].tolist()
ruta_base_celulares = "./BASE CELULARES/BASE CELULARES.xlsx"
df_base_celulares = pd.read_excel(ruta_base_celulares)
deudores_base = df_base_celulares["DEUDOR"].tolist()

no_encontrados = []
for deudor in deudores_base:
    if deudor not in deudores_dacxanalista:
        no_encontrados.append(deudor)
print("Deudores no encontrados:",len(no_encontrados))
print("Deudores encontrados:",len(deudores_base)-len(no_encontrados))

# Valida que los deudores tengan un celular valido
df_base_celulares = df_base_celulares[~df_base_celulares["DEUDOR"].isin(no_encontrados)]
df_base_celulares = df_base_celulares[["DEUDOR", "NOMBRE", "CELULAR"]]
df_base_celulares = df_base_celulares[df_base_celulares["CELULAR"]!=0]
print("Deudores con celular:", df_base_celulares.shape[0])

In [ ]:
df_base_celulares.shape

In [ ]:
df_dacxanalista.to_excel("DACxANALISTA.xlsx",index=False)